In [1]:
import pandas
import numpy as np
import matplotlib.pyplot as plt

%matplotlib inline
%config InlineBackend.figure_format = 'retina'

In [2]:
input_file = 'clean_data.csv'
file = pandas.read_csv(input_file)

In [3]:
from sklearn.model_selection import train_test_split

In [4]:
x = file.Tweet
y = file.Label
seed = 334
# 按（0.98, 0.01, 0.01）的比例划分训练、验证、测试集
x_train, x_validation_test, y_train, y_validation_test = train_test_split(x, y, test_size=0.02, random_state=seed)
x_validation, x_test, y_validation, y_test = train_test_split(x_validation_test, y_validation_test, test_size=0.5, random_state=seed)

# print(len(x))
# print(len(x_train), len(x_val), len(x_test))

| <center><font size=3>Set | <center><font size=3>Size | <center><font size=3>Proportion |
|--|--|--|--|--|--|
| <center><font size=3>Train | <center><font size=3>1565182 | <center><font size=3>98.00% |
| <center><font size=3>Validation | <center><font size=3>15971 | <center><font size=3>1.00% |
| <center><font size=3>Test | <center><font size=3>15982 | <center><font size=3>1.00% |
| <center><font size=3>Total | <center><font size=3>1597125 | <center><font size=3>100.00% |

In [5]:
from gensim.models import KeyedVectors

Using TensorFlow backend.


In [6]:
# 载入预生成的词向量长度为100的word2vec模型
model_cbow = KeyedVectors.load('./word2vec_models/model_cbow_100.word2vec')
model_sg = KeyedVectors.load('./word2vec_models/model_sg_100.word2vec')

# len(model_cbow.wv.vocab)

| <center><font size=3>Vocabulary Size | <center><font size=3>Vector Size| <center><font size=3>Vector Type |
|--|--|--|
| <center><font size=3>268681 | <center><font size=3>100 | <center><font size=3>numpy.ndarray |

In [7]:
from keras.preprocessing.text import Tokenizer

In [8]:
# 创建不同词频阈值的分词器，并将每个句子中的单词序列变为数字序列
num_words=100000

tokenizer_100000 = Tokenizer(num_words=num_words)
# tokenizer_150000 = Tokenizer(num_words=150000)
# tokenizer_200000 = Tokenizer(num_words=200000)
# tokenizer_250000 = Tokenizer(num_words=250000)

tokenizer_100000.fit_on_texts(x_train)
# tokenizer_150000.fit_on_texts(x_train)
# tokenizer_200000.fit_on_texts(x_train)
# tokenizer_250000.fit_on_texts(x_train)

sequences_train_100000 = tokenizer_100000.texts_to_sequences(x_train)
# sequences_train_150000 = tokenizer_150000.texts_to_sequences(x_train)
# sequences_train_200000 = tokenizer_200000.texts_to_sequences(x_train)
# sequences_200000 = tokenizer_250000.texts_to_sequences(x_train)

# 利用训练集上的分词器对验证集和测试集进行序列化
sequences_validation_100000 = tokenizer_100000.texts_to_sequences(x_validation)
sequences_test_100000 = tokenizer_100000.texts_to_sequences(x_test)
# sequences_validation_150000 = tokenizer_150000.texts_to_sequences(x_validation)
# sequences_test_150000 = tokenizer_150000.texts_to_sequences(x_test)
# sequences_validation_200000 = tokenizer_200000.texts_to_sequences(x_validation)
# sequences_test_200000 = tokenizer_200000.texts_to_sequences(x_test)

In [9]:
# 确定句子中最大的单词数
len_max = 0
for x in x_train:
    temp = len(x.split())
    if temp > len_max:
        len_max = temp

# len_max

# 这里不能用sequence的长度来确定句子的最大长度，因为原始数据中存在乱码，如下面的两个print语句所示，而预处理的时候没有考虑到这一点
# print(sequences_100000[510758:510759])
# print(x_train[510758:510759])
# x_train 输入num时，为index=num的元素，输入num1:num2时，为第num1行到第num2行的元素

In [10]:
from keras.preprocessing.sequence import pad_sequences

In [11]:
# 对于变长的句子进行padding使其成为维度相同的标准输入
x_train_pad = pad_sequences(sequences_train_100000, maxlen=70, padding='post')
x_validation_pad = pad_sequences(sequences_validation_100000, maxlen=70, padding='post')
x_test_pad = pad_sequences(sequences_test_100000, maxlen=70, padding='post')

# x_train_pad[510758:510759]

In [12]:
# 创建嵌入矩阵（即在输入神经网络时将标号转为向量的索引）
embedding_matrix_cbow = np.zeros((num_words, 100))
embedding_matrix_sg = np.zeros((num_words, 100))
embedding_matrix_cbow_sg = np.zeros((num_words, 200))
for word, rank in tokenizer_100000.word_index.items():
    if rank >= num_words:
        break
    if word in model_cbow:
        embedding_matrix_cbow[rank] = model_cbow[word]
    if word in model_sg:
        embedding_matrix_sg[rank] = model_sg[word]
    if (word in model_cbow) and (word in model_sg):
        embedding_matrix_cbow_sg[rank] = np.append(model_cbow[word],model_sg[word])

# 某些词如‘quot’等不具有实意的词在word2vec模型中没有对应的词向量，但词频极高，现在的处理会将其视为零向量

# print(x_train[1:2])
# sequences_train_100000[1:2]
# print(embedding_matrix_cbow[297])
# print(model_cbow['play'])

In [13]:
from keras.models import Sequential
from keras.layers import Dense, Conv1D, GlobalMaxPooling1D
from keras.layers.embeddings import Embedding 
from keras.callbacks import TensorBoard
import tensorflow as tf

In [14]:
class TB(TensorBoard):
    def __init__(self, log_every=1, **kwargs):
        super().__init__(**kwargs)
        self.log_every = log_every
        self.counter = 0
    
    def on_batch_end(self, batch, logs=None):
        self.counter+=1
        if self.counter%self.log_every==0:
            for name, value in logs.items():
                if name in ['batch', 'size']:
                    continue
                summary = tf.Summary()
                summary_value = summary.value.add()
                summary_value.simple_value = value.item()
                summary_value.tag = name
                self.writer.add_summary(summary, self.counter)
            self.writer.flush()
        
        super().on_batch_end(batch, logs)
        
# model.fit(params, ..., callbacks[TB(log_dir='...')])

In [17]:
model_cnn_embedding_matrix_cbow = Sequential()
model_cnn_embedding_matrix_cbow.add(Embedding(100000, 100, weights=[embedding_matrix_cbow], input_length=70, trainable=False))
model_cnn_embedding_matrix_cbow.add(Conv1D(filters=100, kernel_size=2, padding='valid', activation='relu', strides=1))
model_cnn_embedding_matrix_cbow.add(GlobalMaxPooling1D())
model_cnn_embedding_matrix_cbow.add(Dense(256, activation='relu'))
model_cnn_embedding_matrix_cbow.add(Dense(128, activation='relu'))
model_cnn_embedding_matrix_cbow.add(Dense(1, activation='sigmoid'))
model_cnn_embedding_matrix_cbow.compile(loss='binary_crossentropy', optimizer='adam', metrics=['acc'])
model_cnn_embedding_matrix_cbow.fit(x_train_pad, y_train, validation_data=(x_validation_pad, y_validation), epochs=5, batch_size=32, verbose=1, callbacks=[TB(log_dir='./tmp/log/model_cnn_embedding_matrix_cbow/')])
model_cnn_embedding_matrix_cbow.save('./models/model_cnn_embedding_matrix_cbow.h5')

Instructions for updating:
`NHWC` for data_format is deprecated, use `NWC` instead
Train on 1565182 samples, validate on 15971 samples
Epoch 1/5
1565182/1565182 [==============================] - 618s 395us/step - loss: 0.4219 - acc: 0.8050 - val_loss: 0.4035 - val_acc: 0.8187
Epoch 2/5
1565182/1565182 [==============================] - 614s 392us/step - loss: 0.3999 - acc: 0.8174 - val_loss: 0.3953 - val_acc: 0.8222
Epoch 3/5
1565182/1565182 [==============================] - 619s 396us/step - loss: 0.3927 - acc: 0.8215 - val_loss: 0.3949 - val_acc: 0.8211
Epoch 4/5
1565182/1565182 [==============================] - 543s 347us/step - loss: 0.3885 - acc: 0.8239 - val_loss: 0.3983 - val_acc: 0.8185
Epoch 5/5
1565182/1565182 [==============================] - 537s 343us/step - loss: 0.3858 - acc: 0.8255 - val_loss: 0.3935 - val_acc: 0.8250


In [18]:
model_cnn_embedding_matrix_sg = Sequential()
model_cnn_embedding_matrix_sg.add(Embedding(100000, 100, weights=[embedding_matrix_sg], input_length=70, trainable=False))
model_cnn_embedding_matrix_sg.add(Conv1D(filters=100, kernel_size=2, padding='valid', activation='relu', strides=1))
model_cnn_embedding_matrix_sg.add(GlobalMaxPooling1D())
model_cnn_embedding_matrix_sg.add(Dense(256, activation='relu'))
model_cnn_embedding_matrix_sg.add(Dense(128, activation='relu'))
model_cnn_embedding_matrix_sg.add(Dense(1, activation='sigmoid'))
model_cnn_embedding_matrix_sg.compile(loss='binary_crossentropy', optimizer='adam', metrics=['acc'])
model_cnn_embedding_matrix_sg.fit(x_train_pad, y_train, validation_data=(x_validation_pad, y_validation), epochs=5, batch_size=32, verbose=1, callbacks=[TB(log_dir='./tmp/log/model_cnn_embedding_matrix_sg/')])
model_cnn_embedding_matrix_sg.save('./models/model_cnn_embedding_matrix_sg.h5')

Train on 1565182 samples, validate on 15971 samples
Epoch 1/5
1565182/1565182 [==============================] - 524s 335us/step - loss: 0.4133 - acc: 0.8100 - val_loss: 0.3961 - val_acc: 0.8192
Epoch 2/5
1565182/1565182 [==============================] - 523s 334us/step - loss: 0.3915 - acc: 0.8221 - val_loss: 0.3930 - val_acc: 0.8239
Epoch 3/5
1565182/1565182 [==============================] - 519s 332us/step - loss: 0.3844 - acc: 0.8260 - val_loss: 0.3891 - val_acc: 0.8249
Epoch 4/5
1565182/1565182 [==============================] - 523s 334us/step - loss: 0.3799 - acc: 0.8286 - val_loss: 0.3887 - val_acc: 0.8260
Epoch 5/5
1565182/1565182 [==============================] - 537s 343us/step - loss: 0.3769 - acc: 0.8302 - val_loss: 0.3918 - val_acc: 0.8259


| <center><font size=3>Embedding Matrix | <center><font size=3>Loss | <center><font size=3>Acc | <center><font size=3>Val Loss | <center><font size=3>Val Acc | <center><font size=3>Time(s) |
|--|--|--|--|--|
| <center><font size=3>CBOW | <center><font size=3>0.3858 | <center><font size=3>0.8255 | <center><font size=3>0.3935 | <center><font size=3>0.8250 | <center><font size=3>2931 |
| <center><font size=3>SG | <center><font size=3>0.3769 | <center><font size=3>0.8302 | <center><font size=3>0.3918 | <center><font size=3>0.8259 | <center><font size=3>2626 |
| <center><font size=3>CBOW+SG | <center><font size=3>0.3787 | <center><font size=3>0.8297 | <center><font size=3>0.3891 | <center><font size=3>0.8266 | <center><font size=3>3427 |
 <center> **Filter Num = 100, Filter Size = 2, Strides = 1, Partition = 1, Epochs = 5

In [19]:
model_cnn_filter_num_100 = Sequential()
model_cnn_filter_num_100.add(Embedding(100000, 200, weights=[embedding_matrix_cbow_sg], input_length=70, trainable=False))
model_cnn_filter_num_100.add(Conv1D(filters=100, kernel_size=2, padding='valid', activation='relu', strides=1))
model_cnn_filter_num_100.add(GlobalMaxPooling1D())
model_cnn_filter_num_100.add(Dense(256, activation='relu'))
model_cnn_filter_num_100.add(Dense(128, activation='relu'))
model_cnn_filter_num_100.add(Dense(1, activation='sigmoid'))
model_cnn_filter_num_100.compile(loss='binary_crossentropy', optimizer='adam', metrics=['acc'])
model_cnn_filter_num_100.fit(x_train_pad, y_train, validation_data=(x_validation_pad, y_validation), epochs=5, batch_size=32, verbose=1, callbacks=[TB(log_dir='./tmp/log/model_cnn_filter_num_100/')])
model_cnn_filter_num_100.save('./models/model_cnn_filter_num_100.h5')

Train on 1565182 samples, validate on 15971 samples
Epoch 1/5
1565182/1565182 [==============================] - 697s 445us/step - loss: 0.4158 - acc: 0.8087 - val_loss: 0.4027 - val_acc: 0.8212
Epoch 2/5
1565182/1565182 [==============================] - 690s 441us/step - loss: 0.3932 - acc: 0.8213 - val_loss: 0.3904 - val_acc: 0.8232
Epoch 3/5
1565182/1565182 [==============================] - 677s 433us/step - loss: 0.3858 - acc: 0.8255 - val_loss: 0.3893 - val_acc: 0.8266
Epoch 4/5
1565182/1565182 [==============================] - 680s 434us/step - loss: 0.3817 - acc: 0.8280 - val_loss: 0.3951 - val_acc: 0.8269
Epoch 5/5
1565182/1565182 [==============================] - 683s 436us/step - loss: 0.3787 - acc: 0.8297 - val_loss: 0.3891 - val_acc: 0.8266


In [20]:
model_cnn_filter_num_50 = Sequential()
model_cnn_filter_num_50.add(Embedding(100000, 200, weights=[embedding_matrix_cbow_sg], input_length=70, trainable=False))
model_cnn_filter_num_50.add(Conv1D(filters=50, kernel_size=2, padding='valid', activation='relu', strides=1))
model_cnn_filter_num_50.add(GlobalMaxPooling1D())
model_cnn_filter_num_50.add(Dense(256, activation='relu'))
model_cnn_filter_num_50.add(Dense(128, activation='relu'))
model_cnn_filter_num_50.add(Dense(1, activation='sigmoid'))
model_cnn_filter_num_50.compile(loss='binary_crossentropy', optimizer='adam', metrics=['acc'])
model_cnn_filter_num_50.fit(x_train_pad, y_train, validation_data=(x_validation_pad, y_validation), epochs=5, batch_size=32, verbose=1, callbacks=[TB(log_dir='./tmp/log/model_cnn_filter_num_50/')])
model_cnn_filter_num_50.save('./models/model_cnn_filter_num_50.h5')

Train on 1565182 samples, validate on 15971 samples
Epoch 1/5
1565182/1565182 [==============================] - 463s 296us/step - loss: 0.4210 - acc: 0.8055 - val_loss: 0.4033 - val_acc: 0.8154
Epoch 2/5
1565182/1565182 [==============================] - 476s 304us/step - loss: 0.4013 - acc: 0.8169 - val_loss: 0.4010 - val_acc: 0.8162
Epoch 3/5
1565182/1565182 [==============================] - 469s 299us/step - loss: 0.3956 - acc: 0.8198 - val_loss: 0.3959 - val_acc: 0.8205
Epoch 4/5
1565182/1565182 [==============================] - 472s 301us/step - loss: 0.3925 - acc: 0.8217 - val_loss: 0.3967 - val_acc: 0.8224
Epoch 5/5
1565182/1565182 [==============================] - 470s 301us/step - loss: 0.3905 - acc: 0.8230 - val_loss: 0.3949 - val_acc: 0.8201


In [21]:
model_cnn_filter_num_150 = Sequential()
model_cnn_filter_num_150.add(Embedding(100000, 200, weights=[embedding_matrix_cbow_sg], input_length=70, trainable=False))
model_cnn_filter_num_150.add(Conv1D(filters=150, kernel_size=2, padding='valid', activation='relu', strides=1))
model_cnn_filter_num_150.add(GlobalMaxPooling1D())
model_cnn_filter_num_150.add(Dense(256, activation='relu'))
model_cnn_filter_num_150.add(Dense(128, activation='relu'))
model_cnn_filter_num_150.add(Dense(1, activation='sigmoid'))
model_cnn_filter_num_150.compile(loss='binary_crossentropy', optimizer='adam', metrics=['acc'])
model_cnn_filter_num_150.fit(x_train_pad, y_train, validation_data=(x_validation_pad, y_validation), epochs=5, batch_size=32, verbose=1, callbacks=[TB(log_dir='./tmp/log/model_cnn_filter_num_150/')])
model_cnn_filter_num_150.save('./models/model_cnn_filter_num_150.h5')

Train on 1565182 samples, validate on 15971 samples
Epoch 1/5
1565182/1565182 [==============================] - 920s 588us/step - loss: 0.4132 - acc: 0.8101 - val_loss: 0.3953 - val_acc: 0.8209
Epoch 2/5
1565182/1565182 [==============================] - 930s 594us/step - loss: 0.3891 - acc: 0.8237 - val_loss: 0.3955 - val_acc: 0.8230
Epoch 3/5
1565182/1565182 [==============================] - 933s 596us/step - loss: 0.3808 - acc: 0.8284 - val_loss: 0.3865 - val_acc: 0.8286
Epoch 4/5
1565182/1565182 [==============================] - 948s 606us/step - loss: 0.3761 - acc: 0.8307 - val_loss: 0.3813 - val_acc: 0.8296
Epoch 5/5
1565182/1565182 [==============================] - 949s 606us/step - loss: 0.3725 - acc: 0.8329 - val_loss: 0.3828 - val_acc: 0.8261


| <center><font size=3>Filter Num | <center><font size=3>Loss | <center><font size=3>Acc | <center><font size=3>Val Loss | <center><font size=3>Val Acc | <center><font size=3>Time(s) |
|--|--|--|--|--|--|
| <center><font size=3>50 | <center><font size=3>0.3905 | <center><font size=3>0.8230 | <center><font size=3>0.3949 | <center><font size=3>0.8201 | <center><font size=3>2350 |
| <center><font size=3>100 | <center><font size=3>0.3787 | <center><font size=3>0.8297 | <center><font size=3>0.3891 | <center><font size=3>0.8266 | <center><font size=3>3427 |
| <center><font size=3>150 | <center><font size=3>0.3725 | <center><font size=3>0.8329 | <center><font size=3>0.3828 | <center><font size=3>0.8261 | <center><font size=3>4680 |
 <center> **Filter Size = 2, Strides = 1, Partition = 1, Epochs = 5, Embedding Matrix = CBOW+SG

In [22]:
model_cnn_filter_size_3 = Sequential()
model_cnn_filter_size_3.add(Embedding(100000, 200, weights=[embedding_matrix_cbow_sg], input_length=70, trainable=False))
model_cnn_filter_size_3.add(Conv1D(filters=100, kernel_size=3, padding='valid', activation='relu', strides=1))
model_cnn_filter_size_3.add(GlobalMaxPooling1D())
model_cnn_filter_size_3.add(Dense(256, activation='relu'))
model_cnn_filter_size_3.add(Dense(128, activation='relu'))
model_cnn_filter_size_3.add(Dense(1, activation='sigmoid'))
model_cnn_filter_size_3.compile(loss='binary_crossentropy', optimizer='adam', metrics=['acc'])
model_cnn_filter_size_3.fit(x_train_pad, y_train, validation_data=(x_validation_pad, y_validation), epochs=5, batch_size=32, verbose=1, callbacks=[TB(log_dir='./tmp/log/model_cnn_filter_size_3')])
model_cnn_filter_size_3.save('./models/model_cnn_filter_size_3.h5')

Train on 1565182 samples, validate on 15971 samples
Epoch 1/5
1565182/1565182 [==============================] - 812s 519us/step - loss: 0.4142 - acc: 0.8097 - val_loss: 0.3955 - val_acc: 0.8176
Epoch 2/5
1565182/1565182 [==============================] - 808s 516us/step - loss: 0.3897 - acc: 0.8238 - val_loss: 0.3860 - val_acc: 0.8261
Epoch 3/5
1565182/1565182 [==============================] - 817s 522us/step - loss: 0.3817 - acc: 0.8282 - val_loss: 0.3818 - val_acc: 0.8283
Epoch 4/5
1565182/1565182 [==============================] - 831s 531us/step - loss: 0.3767 - acc: 0.8308 - val_loss: 0.3798 - val_acc: 0.8309
Epoch 5/5
1565182/1565182 [==============================] - 837s 535us/step - loss: 0.3730 - acc: 0.8329 - val_loss: 0.3827 - val_acc: 0.8269


In [23]:
model_cnn_filter_size_5 = Sequential()
model_cnn_filter_size_5.add(Embedding(100000, 200, weights=[embedding_matrix_cbow_sg], input_length=70, trainable=False))
model_cnn_filter_size_5.add(Conv1D(filters=100, kernel_size=5, padding='valid', activation='relu', strides=1))
model_cnn_filter_size_5.add(GlobalMaxPooling1D())
model_cnn_filter_size_5.add(Dense(256, activation='relu'))
model_cnn_filter_size_5.add(Dense(128, activation='relu'))
model_cnn_filter_size_5.add(Dense(1, activation='sigmoid'))
model_cnn_filter_size_5.compile(loss='binary_crossentropy', optimizer='adam', metrics=['acc'])
model_cnn_filter_size_5.fit(x_train_pad, y_train, validation_data=(x_validation_pad, y_validation), epochs=5, batch_size=32, verbose=1, callbacks=[TB(log_dir='./tmp/log/model_cnn_filter_size_5')])
model_cnn_filter_size_5.save('./models/model_cnn_filter_size_5.h5')

Train on 1565182 samples, validate on 15971 samples
Epoch 1/5
1565182/1565182 [==============================] - 1075s 687us/step - loss: 0.4162 - acc: 0.8088 - val_loss: 0.3974 - val_acc: 0.8154
Epoch 2/5
1565182/1565182 [==============================] - 1052s 672us/step - loss: 0.3893 - acc: 0.8237 - val_loss: 0.3940 - val_acc: 0.8261
Epoch 3/5
1565182/1565182 [==============================] - 1048s 669us/step - loss: 0.3796 - acc: 0.8295 - val_loss: 0.3864 - val_acc: 0.8240
Epoch 4/5
1565182/1565182 [==============================] - 1059s 677us/step - loss: 0.3733 - acc: 0.8326 - val_loss: 0.3884 - val_acc: 0.8252
Epoch 5/5
1565182/1565182 [==============================] - 1069s 683us/step - loss: 0.3689 - acc: 0.8350 - val_loss: 0.3914 - val_acc: 0.8257


| <center><font size=3>Filter Size | <center><font size=3>Loss | <center><font size=3>Acc | <center><font size=3>Val Loss | <center><font size=3>Val Acc | <center><font size=3>Time(s) |
|--|--|--|--|--|--|
| <center><font size=3>2 | <center><font size=3>0.3787 | <center><font size=3>0.8297 | <center><font size=3>0.3891 | <center><font size=3>0.8266 | <center><font size=3>3427 |
| <center><font size=3>3 | <center><font size=3>0.3730 | <center><font size=3>0.8329 | <center><font size=3>0.3827 | <center><font size=3>0.8269 | <center><font size=3>4105 |
| <center><font size=3>5 | <center><font size=3>0.3689 | <center><font size=3>0.8350 | <center><font size=3>0.3914 | <center><font size=3>0.8257 | <center><font size=3>5303 |
 <center> **Filter Num = 100, Strides = 1, Partition = 1, Epochs = 5, Embedding Matrix = CBOW+SG

In [19]:
model_cnn_stride_2 = Sequential()
model_cnn_stride_2.add(Embedding(100000, 200, weights=[embedding_matrix_cbow_sg], input_length=70, trainable=False))
model_cnn_stride_2.add(Conv1D(filters=100, kernel_size=2, padding='valid', activation='relu', strides=2))
model_cnn_stride_2.add(GlobalMaxPooling1D())
model_cnn_stride_2.add(Dense(256, activation='relu'))
model_cnn_stride_2.add(Dense(128, activation='relu'))
model_cnn_stride_2.add(Dense(1, activation='sigmoid'))
model_cnn_stride_2.compile(loss='binary_crossentropy', optimizer='adam', metrics=['acc'])
model_cnn_stride_2.fit(x_train_pad, y_train, validation_data=(x_validation_pad, y_validation), epochs=5, batch_size=32, verbose=1, callbacks=[TB(log_dir='./tmp/log/model_cnn_stride_2/')])
model_cnn_stride_2.save('./models/model_cnn_stride_2.h5')

Instructions for updating:
`NHWC` for data_format is deprecated, use `NWC` instead
Train on 1565182 samples, validate on 15971 samples
Epoch 1/5
1565182/1565182 [==============================] - 528s 338us/step - loss: 0.4305 - acc: 0.7995 - val_loss: 0.4142 - val_acc: 0.8097
Epoch 2/5
1565182/1565182 [==============================] - 587s 375us/step - loss: 0.4098 - acc: 0.8116 - val_loss: 0.4210 - val_acc: 0.8104
Epoch 3/5
1565182/1565182 [==============================] - 587s 375us/step - loss: 0.4029 - acc: 0.8153 - val_loss: 0.4098 - val_acc: 0.8138
Epoch 4/5
1565182/1565182 [==============================] - 606s 387us/step - loss: 0.3984 - acc: 0.8181 - val_loss: 0.4073 - val_acc: 0.8161
Epoch 5/5
1565182/1565182 [==============================] - 551s 352us/step - loss: 0.3958 - acc: 0.8194 - val_loss: 0.4037 - val_acc: 0.8183


In [20]:
model_cnn_stride_3 = Sequential()
model_cnn_stride_3.add(Embedding(100000, 200, weights=[embedding_matrix_cbow_sg], input_length=70, trainable=False))
model_cnn_stride_3.add(Conv1D(filters=100, kernel_size=2, padding='valid', activation='relu', strides=3))
model_cnn_stride_3.add(GlobalMaxPooling1D())
model_cnn_stride_3.add(Dense(256, activation='relu'))
model_cnn_stride_3.add(Dense(128, activation='relu'))
model_cnn_stride_3.add(Dense(1, activation='sigmoid'))
model_cnn_stride_3.compile(loss='binary_crossentropy', optimizer='adam', metrics=['acc'])
model_cnn_stride_3.fit(x_train_pad, y_train, validation_data=(x_validation_pad, y_validation), epochs=5, batch_size=32, verbose=1, callbacks=[TB(log_dir='./tmp/log/model_cnn_stride_3/')])
model_cnn_stride_3.save('./models/model_cnn_stride_3.h5')

Train on 1565182 samples, validate on 15971 samples
Epoch 1/5
1565182/1565182 [==============================] - 3691s 2ms/step - loss: 0.4820 - acc: 0.7663 - val_loss: 0.4627 - val_acc: 0.7756
Epoch 2/5
1565182/1565182 [==============================] - 349s 223us/step - loss: 0.4634 - acc: 0.7782 - val_loss: 0.4587 - val_acc: 0.7817
Epoch 3/5
1565182/1565182 [==============================] - 365s 233us/step - loss: 0.4571 - acc: 0.7821 - val_loss: 0.4565 - val_acc: 0.7821
Epoch 4/5
1565182/1565182 [==============================] - 374s 239us/step - loss: 0.4535 - acc: 0.7845 - val_loss: 0.4546 - val_acc: 0.7834
Epoch 5/5
1565182/1565182 [==============================] - 390s 249us/step - loss: 0.4506 - acc: 0.7862 - val_loss: 0.4565 - val_acc: 0.7819


| <center><font size=3>Stride | <center><font size=3>Loss | <center><font size=3>Acc | <center><font size=3>Val Loss | <center><font size=3>Val Acc | <center><font size=3>Time(s) |
|--|--|--|--|--|--|
| <center><font size=3>1 | <center><font size=3>0.3787 | <center><font size=3>0.8297 | <center><font size=3>0.3891 | <center><font size=3>0.8266 | <center><font size=3>3427 |
| <center><font size=3>2 | <center><font size=3>0.3958 | <center><font size=3>0.8194 | <center><font size=3>0.4037 | <center><font size=3>0.8183 | <center><font size=3>2859 |
| <center><font size=3>3 | <center><font size=3>0.4506 | <center><font size=3>0.7862 | <center><font size=3>0.4565 | <center><font size=3>0.7819 | <center><font size=3>1823 |
 <center> **Filter Size = 2, Filter Num = 100, Partition = 1, Epochs = 5, Embedding Matrix = CBOW+SG

In [20]:
from keras.layers import MaxPooling1D, Flatten

In [21]:
model_cnn_2_partition_pooling = Sequential()
model_cnn_2_partition_pooling.add(Embedding(100000, 200, weights=[embedding_matrix_cbow_sg], input_length=70, trainable=False))
model_cnn_2_partition_pooling.add(Conv1D(filters=100, kernel_size=2, padding='valid', activation='relu', strides=1))
model_cnn_2_partition_pooling.add(MaxPooling1D(pool_size=35))
model_cnn_2_partition_pooling.add(Flatten())
model_cnn_2_partition_pooling.add(Dense(256, activation='relu'))
model_cnn_2_partition_pooling.add(Dense(128, activation='relu'))
model_cnn_2_partition_pooling.add(Dense(1, activation='sigmoid'))
model_cnn_2_partition_pooling.compile(loss='binary_crossentropy', optimizer='adam', metrics=['acc'])
model_cnn_2_partition_pooling.fit(x_train_pad, y_train, validation_data=(x_validation_pad, y_validation), epochs=5, batch_size=32, verbose=1, callbacks=[TB(log_dir='./tmp/log/model_cnn_2_partition_pooling')])
model_cnn_2_partition_pooling.save('./models/model_cnn_2_partition_pooling.h5')

Train on 1565182 samples, validate on 15971 samples
Epoch 1/5
1565182/1565182 [==============================] - 515s 329us/step - loss: 0.4154 - acc: 0.8085 - val_loss: 0.3961 - val_acc: 0.8198
Epoch 2/5
1565182/1565182 [==============================] - 612s 391us/step - loss: 0.3928 - acc: 0.8215 - val_loss: 0.3888 - val_acc: 0.8223
Epoch 3/5
1565182/1565182 [==============================] - 642s 410us/step - loss: 0.3858 - acc: 0.8256 - val_loss: 0.3843 - val_acc: 0.8278
Epoch 4/5
1565182/1565182 [==============================] - 584s 373us/step - loss: 0.3814 - acc: 0.8281 - val_loss: 0.3834 - val_acc: 0.8258
Epoch 5/5
1565182/1565182 [==============================] - 583s 373us/step - loss: 0.3785 - acc: 0.8296 - val_loss: 0.3840 - val_acc: 0.8271


In [22]:
model_cnn_3_partition_pooling = Sequential()
model_cnn_3_partition_pooling.add(Embedding(100000, 200, weights=[embedding_matrix_cbow_sg], input_length=70, trainable=False))
model_cnn_3_partition_pooling.add(Conv1D(filters=100, kernel_size=2, padding='valid', activation='relu', strides=1))
model_cnn_3_partition_pooling.add(MaxPooling1D(pool_size=24))
model_cnn_3_partition_pooling.add(Flatten())
model_cnn_3_partition_pooling.add(Dense(256, activation='relu'))
model_cnn_3_partition_pooling.add(Dense(128, activation='relu'))
model_cnn_3_partition_pooling.add(Dense(1, activation='sigmoid'))
model_cnn_3_partition_pooling.compile(loss='binary_crossentropy', optimizer='adam', metrics=['acc'])
model_cnn_3_partition_pooling.fit(x_train_pad, y_train, validation_data=(x_validation_pad, y_validation), epochs=5, batch_size=32, verbose=1, callbacks=[TB(log_dir='./tmp/log/model_cnn_3_partition_pooling')])
model_cnn_3_partition_pooling.save('./models/model_cnn_3_partition_pooling.h5')

Train on 1565182 samples, validate on 15971 samples
Epoch 1/5
1565182/1565182 [==============================] - 618s 395us/step - loss: 0.4155 - acc: 0.8088 - val_loss: 0.3955 - val_acc: 0.8211
Epoch 2/5
1565182/1565182 [==============================] - 582s 372us/step - loss: 0.3929 - acc: 0.8216 - val_loss: 0.3906 - val_acc: 0.8212
Epoch 3/5
1565182/1565182 [==============================] - 634s 405us/step - loss: 0.3855 - acc: 0.8256 - val_loss: 0.3847 - val_acc: 0.8244
Epoch 4/5
1565182/1565182 [==============================] - 621s 397us/step - loss: 0.3811 - acc: 0.8285 - val_loss: 0.3855 - val_acc: 0.8295
Epoch 5/5
1565182/1565182 [==============================] - 489s 313us/step - loss: 0.3778 - acc: 0.8300 - val_loss: 0.3818 - val_acc: 0.8279


In [23]:
model_cnn_4_partition_pooling = Sequential()
model_cnn_4_partition_pooling.add(Embedding(100000, 200, weights=[embedding_matrix_cbow_sg], input_length=70, trainable=False))
model_cnn_4_partition_pooling.add(Conv1D(filters=100, kernel_size=2, padding='valid', activation='relu', strides=1))
model_cnn_4_partition_pooling.add(MaxPooling1D(pool_size=18))
model_cnn_4_partition_pooling.add(Flatten())
model_cnn_4_partition_pooling.add(Dense(256, activation='relu'))
model_cnn_4_partition_pooling.add(Dense(128, activation='relu'))
model_cnn_4_partition_pooling.add(Dense(1, activation='sigmoid'))
model_cnn_4_partition_pooling.compile(loss='binary_crossentropy', optimizer='adam', metrics=['acc'])
model_cnn_4_partition_pooling.fit(x_train_pad, y_train, validation_data=(x_validation_pad, y_validation), epochs=5, batch_size=32, verbose=1, callbacks=[TB(log_dir='./tmp/log/model_cnn_4_partition_pooling')])
model_cnn_4_partition_pooling.save('./models/model_cnn_4_partition_pooling.h5')

Train on 1565182 samples, validate on 15971 samples
Epoch 1/5
1565182/1565182 [==============================] - 487s 311us/step - loss: 0.4145 - acc: 0.8095 - val_loss: 0.3935 - val_acc: 0.8216
Epoch 2/5
1565182/1565182 [==============================] - 519s 331us/step - loss: 0.3924 - acc: 0.8220 - val_loss: 0.3884 - val_acc: 0.8242
Epoch 3/5
1565182/1565182 [==============================] - 508s 325us/step - loss: 0.3852 - acc: 0.8262 - val_loss: 0.3851 - val_acc: 0.8266
Epoch 4/5
1565182/1565182 [==============================] - 508s 325us/step - loss: 0.3812 - acc: 0.8281 - val_loss: 0.3804 - val_acc: 0.8294
Epoch 5/5
1565182/1565182 [==============================] - 496s 317us/step - loss: 0.3782 - acc: 0.8299 - val_loss: 0.3825 - val_acc: 0.8291


In [24]:
model_cnn_5_partition_pooling = Sequential()
model_cnn_5_partition_pooling.add(Embedding(100000, 200, weights=[embedding_matrix_cbow_sg], input_length=70, trainable=False))
model_cnn_5_partition_pooling.add(Conv1D(filters=100, kernel_size=2, padding='valid', activation='relu', strides=1))
model_cnn_5_partition_pooling.add(MaxPooling1D(pool_size=14))
model_cnn_5_partition_pooling.add(Flatten())
model_cnn_5_partition_pooling.add(Dense(256, activation='relu'))
model_cnn_5_partition_pooling.add(Dense(128, activation='relu'))
model_cnn_5_partition_pooling.add(Dense(1, activation='sigmoid'))
model_cnn_5_partition_pooling.compile(loss='binary_crossentropy', optimizer='adam', metrics=['acc'])
model_cnn_5_partition_pooling.fit(x_train_pad, y_train, validation_data=(x_validation_pad, y_validation), epochs=5, batch_size=32, verbose=1, callbacks=[TB(log_dir='./tmp/log/model_cnn_5_partition_pooling')])
model_cnn_5_partition_pooling.save('./models/model_cnn_5_partition_pooling.h5')

Train on 1565182 samples, validate on 15971 samples
Epoch 1/5
1565182/1565182 [==============================] - 512s 327us/step - loss: 0.4143 - acc: 0.8097 - val_loss: 0.3954 - val_acc: 0.8215
Epoch 2/5
1565182/1565182 [==============================] - 494s 316us/step - loss: 0.3915 - acc: 0.8227 - val_loss: 0.3885 - val_acc: 0.8243
Epoch 3/5
1565182/1565182 [==============================] - 499s 319us/step - loss: 0.3837 - acc: 0.8270 - val_loss: 0.3874 - val_acc: 0.8237
Epoch 4/5
1565182/1565182 [==============================] - 510s 326us/step - loss: 0.3791 - acc: 0.8292 - val_loss: 0.3838 - val_acc: 0.8278
Epoch 5/5
1565182/1565182 [==============================] - 518s 331us/step - loss: 0.3762 - acc: 0.8312 - val_loss: 0.3880 - val_acc: 0.8252


| <center><font size=3>Partition | <center><font size=3>Loss | <center><font size=3>Acc | <center><font size=3>Val Loss | <center><font size=3>Val Acc | <center><font size=3>Time(s) |
|--|--|--|--|--|--|
| <center><font size=3>1 | <center><font size=3>0.3787 | <center><font size=3>0.8297 | <center><font size=3>0.3891 | <center><font size=3>0.8266 | <center><font size=3>3427 |
| <center><font size=3>2 | <center><font size=3>0.3758 | <center><font size=3>0.8296 | <center><font size=3>0.3840 | <center><font size=3>0.8271 | <center><font size=3>2936 |
| <center><font size=3>3 | <center><font size=3>0.3778 | <center><font size=3>0.8300 | <center><font size=3>0.3818 | <center><font size=3>0.8279 | <center><font size=3>2944 |
| <center><font size=3>4 | <center><font size=3>0.3782 | <center><font size=3>0.8299 | <center><font size=3>0.3825 | <center><font size=3>0.8291 | <center><font size=3>2516 |
| <center><font size=3>5 | <center><font size=3>0.3762 | <center><font size=3>0.8312 | <center><font size=3>0.3880 | <center><font size=3>0.8252 | <center><font size=3>2533 |
 <center> **Filter Size = 2, Filter Num = 100, Stride = 1, Epochs = 5, Embedding Matrix = CBOW+SG

In [ ]:
from keras import backend as K 
from keras.layers import Lambda

In [ ]:
def k_max(x):
    x.top_k
    
def k_max_output_shape(input_shape, k):
    shape = list(input_shape)
    assert len(shape) == 3
    shape[-1] = k
    return tuple(shape)

In [ ]:
model_cnn_2_max_pooling = Sequential()
model_cnn_2_max_pooling.add(Embedding(100000, 200, weights=[embedding_matrix_cbow_sg], input_length=70, trainable=False))
model_cnn_2_max_pooling.add(Conv1D(filters=100, kernel_size=2, padding='valid', activation='relu', strides=1))
model_cnn_2_max_pooling.add(Lambda(k_max, output_shape=k_max_output_shape(2)))
model_cnn_2_max_pooling.add(Flatten())
model_cnn_2_max_pooling.add(Dense(256, activation='relu'))
model_cnn_2_max_pooling.add(Dense(128, activation='relu'))
model_cnn_2_max_pooling.add(Dense(1, activation='sigmoid'))
model_cnn_2_max_pooling.compile(loss='binary_crossentropy', optimizer='adam', metrics=['acc'])
model_cnn_2_max_pooling.fit(x_train_pad, y_train, validation_data=(x_validation_pad, y_validation), epochs=5, batch_size=32, verbose=1, callbacks=[TB(log_dir='./tmp/log/model_cnn_2_max_pooling')])
model_cnn_2_max_pooling.save('./models/model_cnn_2_max_pooling.h5')

In [ ]:
model_cnn_trainable_true = Sequential()
model_cnn_trainable_true.add(Embedding(100000, 200, weights=[embedding_matrix_cbow_sg], input_length=70, trainable=True))
model_cnn_trainable_true.add(Conv1D(filters=100, kernel_size=2, padding='valid', activation='relu', strides=1))
model_cnn_trainable_true.add(GlobalMaxPooling1D())
model_cnn_trainable_true.add(Dense(256, activation='relu'))
model_cnn_trainable_true.add(Dense(128, activation='relu'))
model_cnn_trainable_true.add(Dense(1, activation='sigmoid'))
model_cnn_trainable_true.compile(loss='binary_crossentropy', optimizer='adam', metrics=['acc'])
model_cnn_trainable_true.fit(x_train_pad, y_train, validation_data=(x_validation_pad, y_validation), epochs=5, batch_size=32, verbose=1, callbacks=[TB(log_dir='./tmp/log/model_cnn_trainable_true/')])
model_cnn_trainable_true.save('./models/model_cnn_trainable_true.h5')

| <center><font size=3>Trainable | <center><font size=3>Loss | <center><font size=3>Acc | <center><font size=3>Val Loss | <center><font size=3>Val Acc | <center><font size=3>Time(s) |
|--|--|--|--|--|--|
| <center><font size=3>False | <center><font size=3>0.3787 | <center><font size=3>0.8297 | <center><font size=3>0.3891 | <center><font size=3>0.8266 | <center><font size=3>3427 |
| <center><font size=3>True | <center><font size=3>0. | <center><font size=3>0. | <center><font size=3>0. | <center><font size=3>0. | <center><font size=3> |
 <center> **Filter Size = 2, Filter Num = 100, Strides = 1, Partition = 1, Epochs = 5, Embedding Matrix = CBOW+SG

In [50]:
from keras.layers import Conv2D, MaxPooling2D, Reshape

In [ ]:
model_cnn_conv2d = Sequential()
model_cnn_conv2d.add(Embedding(100000, 200, weights=[embedding_matrix_cbow_sg], input_length=70, trainable=False))
model_cnn_conv2d.add(Reshape((-1, 200, 1)))
model_cnn_conv2d.add(Conv2D(filters=70, kernel_size=(2, 2), padding='valid', activation='relu', strides=(1, 1)))
model_cnn_conv2d.add(MaxPooling2D(pool_size=(2, 2), padding='valid'))
model_cnn_conv2d.add(Conv2D(filters=50, kernel_size=(2, 2), padding='valid', activation='relu', strides=(1, 1)))
model_cnn_conv2d.add(MaxPooling2D(pool_size=(2, 2), padding='valid'))
model_cnn_conv2d.add(Conv2D(filters=30, kernel_size=(2, 2), padding='valid', activation='relu', strides=(1, 1)))
model_cnn_conv2d.add(MaxPooling2D(pool_size=(2, 2), padding='valid'))
model_cnn_conv2d.add(Flatten())
model_cnn_conv2d.add(Dense(256, activation='relu'))
model_cnn_conv2d.add(Dense(128, activation='relu'))
model_cnn_conv2d.add(Dense(1, activation='sigmoid'))
model_cnn_conv2d.compile(loss='binary_crossentropy', optimizer='adam', metrics=['acc'])
model_cnn_conv2d.fit(x_train_pad, y_train, validation_data=(x_validation_pad, y_validation), epochs=5, batch_size=32, verbose=1, callbacks=[TB(log_dir='./tmp/log/model_cnn_conv2d/')])
model_cnn_conv2d.save('./models/model_cnn_conv2d.h5')

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_21 (Embedding)     (None, 70, 200)           20000000  
_________________________________________________________________
reshape_14 (Reshape)         (None, 70, 200, 1)        0         
_________________________________________________________________
conv2d_32 (Conv2D)           (None, 69, 199, 70)       350       
_________________________________________________________________
max_pooling2d_28 (MaxPooling (None, 34, 99, 70)        0         
_________________________________________________________________
conv2d_33 (Conv2D)           (None, 33, 98, 50)        14050     
_________________________________________________________________
max_pooling2d_29 (MaxPooling (None, 16, 49, 50)        0         
_________________________________________________________________
conv2d_34 (Conv2D)           (None, 15, 48, 30)        6030      
__________

| <center><font size=3>Network | <center><font size=3>Loss | <center><font size=3>Acc | <center><font size=3>Val Loss | <center><font size=3>Val Acc | <center><font size=3>Time(s) |
|--|--|--|--|--|--|
| <center><font size=3>Conv1D | <center><font size=3>0.3787 | <center><font size=3>0.8297 | <center><font size=3>0.3891 | <center><font size=3>0.8266 | <center><font size=3>3427 |
| <center><font size=3>Conv2D | <center><font size=3>0. | <center><font size=3>0. | <center><font size=3>0. | <center><font size=3>0. | <center><font size=3> |
 <center> **Epochs = 5, Embedding Matrix = CBOW+SG

In [15]:
from keras.layers import Input, Dense, concatenate, Activation
from keras.models import Model

In [ ]:
input_type = Input(shape=(70,), dtype='float64')
tweet_encoder = Embedding(100000, 200, weights=[embedding_matrix_cbow_sg], input_length=70, trainable=False)(input_type)
model_2 = Conv1D(filters=50, kernel_size=2, padding='valid', activation='relu', strides=1)(tweet_encoder)
model_2 = GlobalMaxPooling1D()(model_2)
model_3 = Conv1D(filters=50, kernel_size=3, padding='valid', activation='relu', strides=1)(tweet_encoder)
model_3 = GlobalMaxPooling1D()(model_3)
model_5 = Conv1D(filters=50, kernel_size=5, padding='valid', activation='relu', strides=1)(tweet_encoder)
model_5 = GlobalMaxPooling1D()(model_5)
merged = concatenate([model_2, model_3, model_5], axis=1)
merged = Dense(256, activation='relu')(merged)
merged = Dense(128, activation='relu')(merged)
output = Dense(1, activation='sigmoid')(merged)
model_cnn_mixed = Model(inputs=[input_type], outputs=[output])
model_cnn_mixed.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
model_cnn_mixed.fit(x_train_pad, y_train, validation_data=(x_validation_pad, y_validation), epochs=10, batch_size=32, verbose=1, callbacks=[TB(log_dir='./tmp/log/model_cnn_mixed/')])
model_cnn_mixed.save('./models/model_cnn_mixed.h5')

Train on 1565182 samples, validate on 15971 samples
Epoch 1/10
 345056/1565182 [=====>........................] - ETA: 23:14 - loss: 0.4370 - acc: 0.7967

In [ ]:
model_mix.evaluate(x=x_test_pad, y=y_test)

In [ ]:
from sklearn.metrics import roc_curve, auc
FPR, TPR, threshold = roc_curve(y_test, model_mix.predict(x_test_pad))
roc_auc = auc(FPR, TPR)
plt.plot(FPR, TPR)
plt.plot([0, 1], [0, 1], 'k--')
plt.xlabel('False Positive Rate', fontsize=18)
plt.ylabel('True Positive Rate', fontsize=18)
plt.show()